# Plagiarism Detection Model

Now that you've created training and test data, you are ready to define and train a model. Your goal in this notebook, will be to train a binary classification model that learns to label an answer file as either plagiarized or not, based on the features you provide the model.

This task will be broken down into a few discrete steps:

* Upload your data to S3.
* Define a binary classification model and a training script.
* Train your model and deploy it.
* Evaluate your deployed classifier and answer some questions about your approach.

To complete this notebook, you'll have to complete all given exercises and answer all the questions in this notebook.
> All your tasks will be clearly labeled **EXERCISE** and questions as **QUESTION**.

It will be up to you to explore different classification models and decide on a model that gives you the best performance for this dataset.

---

## Load Data to S3

In the last notebook, you should have created two files: a `training.csv` and `test.csv` file with the features and class labels for the given corpus of plagiarized/non-plagiarized text data. 

>The below cells load in some AWS SageMaker libraries and creates a default bucket. After creating this bucket, you can upload your locally stored data to S3.

Save your train and test `.csv` feature files, locally. To do this you can run the second notebook "2_Plagiarism_Feature_Engineering" in SageMaker or you can manually upload your files to this notebook using the upload icon in Jupyter Lab. Then you can upload local files to S3 by using `sagemaker_session.upload_data` and pointing directly to where the training data is saved.

In [1]:
import pandas as pd
import boto3
import sagemaker

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

## EXERCISE: Upload your training data to S3

Specify the `data_dir` where you've saved your `train.csv` file. Decide on a descriptive `prefix` that defines where your data will be uploaded in the default S3 bucket. Finally, create a pointer to your training data by calling `sagemaker_session.upload_data` and passing in the required parameters. It may help to look at the [Session documentation](https://sagemaker.readthedocs.io/en/stable/session.html#sagemaker.session.Session.upload_data) or previous SageMaker code examples.

You are expected to upload your entire directory. Later, the training script will only access the `train.csv` file.

In [3]:
# should be the name of directory you created to save your features data
data_dir = 'plagiarism_data'

# set prefix, a descriptive name for a directory  
prefix = 'plagiarism/train'

# upload all data to S3
bucket_url = sagemaker_session.upload_data(path = data_dir, bucket = bucket, key_prefix = prefix)
bucket_url

's3://sagemaker-us-east-1-476375380884/plagiarism/train'

### Test cell

Test that your data has been successfully uploaded. The below cell prints out the items in your S3 bucket and will throw an error if it is empty. You should see the contents of your `data_dir` and perhaps some checkpoints. If you see any other files listed, then you may have some old model files that you can delete via the S3 console (though, additional files shouldn't affect the performance of model developed in this notebook).

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# confirm that data is in S3 bucket
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('Test passed!')

plagiarism/train/.ipynb_checkpoints/test-checkpoint.csv
plagiarism/train/.ipynb_checkpoints/train-checkpoint.csv
plagiarism/train/test.csv
plagiarism/train/train.csv
Test passed!


---

# Modeling

Now that you've uploaded your training data, it's time to define and train a model!

The type of model you create is up to you. For a binary classification task, you can choose to go one of three routes:
* Use a built-in classification algorithm, like LinearLearner.
* Define a custom Scikit-learn classifier, a comparison of models can be found [here](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html).
* Define a custom PyTorch neural network classifier. 

It will be up to you to test out a variety of models and choose the best one. Your project will be graded on the accuracy of your final model. 
 
---

## EXERCISE: Complete a training script 

To implement a custom classifier, you'll need to complete a `train.py` script. You've been given the folders `source_sklearn` and `source_pytorch` which hold starting code for a custom Scikit-learn model and a PyTorch model, respectively. Each directory has a `train.py` training script. To complete this project **you only need to complete one of these scripts**; the script that is responsible for training your final model.

A typical training script:
* Loads training data from a specified directory
* Parses any training & model hyperparameters (ex. nodes in a neural network, training epochs, etc.)
* Instantiates a model of your design, with any specified hyperparams
* Trains that model 
* Finally, saves the model so that it can be hosted/deployed, later

### Defining and training a model
Much of the training script code is provided for you. Almost all of your work will be done in the `if __name__ == '__main__':` section. To complete a `train.py` file, you will:
1. Import any extra libraries you need
2. Define any additional model training hyperparameters using `parser.add_argument`
2. Define a model in the `if __name__ == '__main__':` section
3. Train the model in that same section

Below, you can use `!pygmentize` to display an existing `train.py` file. Read through the code; all of your tasks are marked with `TODO` comments. 

**Note: If you choose to create a custom PyTorch model, you will be responsible for defining the model in the `model.py` file,** and a `predict.py` file is provided. If you choose to use Scikit-learn, you only need a `train.py` file; you may import a classifier from the `sklearn` library.

In [5]:
# directory can be changed to: source_sklearn or source_pytorch
!pygmentize source_pytorch/train.py

import argparse
import json
import os
import pandas as pd
import torch
import torch.optim as optim
import torch.utils.data

# imports the model in model.py by name
from model import BinaryClassifier

def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BinaryClassifier(model_info['input_features'], model_info['hidden_dim'], model_info['output_dim'])

    # Load the stored model parameters.
    model_path = os.path.join(model_dir, 'model.pth')
    with open(model_path, 'rb') as f:
        model.load_state_dict(torch.load(

In [6]:
!pygmentize source_pytorch/model.py

# torch imports
import torch.nn.functional as F
import torch.nn as nn


## TODO: Complete this classifier
class BinaryClassifier(nn.Module):
    """
    Define a neural network that performs binary classification.
    The network should accept your number of features as input, and produce 
    a single sigmoid value, that can be rounded to a label: 0 or 1, as output.
    
    Notes on training:
    To train a binary classifier in PyTorch, use BCELoss.
    BCELoss is binary cross entropy loss, documentation: https://pytorch.org/docs/stable/nn.html#torch.nn.BCELoss
    """

    ## TODO: Define the init function, the input params are required (for loading code in train.py to work)
    def __init__(self, input_features, hidden_dim, output_dim):
        """
        Initialize the model by setting up linear layers.
        Use the input parameters to help define the layers of your model.
        :param input_features: the number of input features in your training/test data
        :param hid

### Provided code

If you read the code above, you can see that the starter code includes a few things:
* Model loading (`model_fn`) and saving code
* Getting SageMaker's default hyperparameters
* Loading the training data by name, `train.csv` and extracting the features and labels, `train_x`, and `train_y`

If you'd like to read more about model saving with [joblib for sklearn](https://scikit-learn.org/stable/modules/model_persistence.html) or with [torch.save](https://pytorch.org/tutorials/beginner/saving_loading_models.html), click on the provided links.

---
# Create an Estimator

When a custom model is constructed in SageMaker, an entry point must be specified. This is the Python file which will be executed when the model is trained; the `train.py` function you specified above. To run a custom training script in SageMaker, construct an estimator, and fill in the appropriate constructor arguments:

* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `source_sklearn` OR `source_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **role**: Role ARN, which was specified, above.
* **train_instance_count**: The number of training instances (should be left at 1).
* **train_instance_type**: The type of SageMaker instance for training. Note: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types.
* **sagemaker_session**: The session used to train on Sagemaker.
* **hyperparameters** (optional): A dictionary `{'name':value, ..}` passed to the train function as hyperparameters.

Note: For a PyTorch model, there is another optional argument **framework_version**, which you can set to the latest version of PyTorch, `1.0`.

## EXERCISE: Define a Scikit-learn or PyTorch estimator

To import your desired estimator, use one of the following lines:
```
from sagemaker.sklearn.estimator import SKLearn
```
```
from sagemaker.pytorch import PyTorch
```

In [7]:
# your import and estimator code, here

from sagemaker.pytorch import PyTorch

hp = {'epochs': 300, 'hidden_dim': 200}

estimator = PyTorch(entry_point = "train.py",
                    source_dir = "source_pytorch",
                    role = role,
                    framework_version = '0.4.0',
                    train_instance_count = 1,
                    train_instance_type = 'ml.p2.xlarge',
                    hyperparameters = hp)

## EXERCISE: Train the estimator

Train your estimator on the training data stored in S3. This should create a training job that you can monitor in your SageMaker console.

In [8]:
%%time

# Train your estimator on S3 training data

estimator.fit({'training': bucket_url})


2020-02-10 08:45:29 Starting - Starting the training job...
2020-02-10 08:45:30 Starting - Launching requested ML instances......
2020-02-10 08:46:37 Starting - Preparing the instances for training.........
2020-02-10 08:48:18 Downloading - Downloading input data...
2020-02-10 08:48:42 Training - Downloading the training image.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-02-10 08:49:03,010 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-02-10 08:49:03,036 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-02-10 08:49:09,254 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-02-10 08:49:09,587 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-02-10 08:49:09,587 sagemaker-containers INFO     Generating setup.cfg
2020-02-10 08:49:09,587 sagemaker-containers

## EXERCISE: Deploy the trained model

After training, deploy your model to create a `predictor`. If you're using a PyTorch model, you'll need to create a trained `PyTorchModel` that accepts the trained `<model>.model_data` as an input parameter and points to the provided `source_pytorch/predict.py` file as an entry point. 

To deploy a trained model, you'll use `<model>.deploy`, which takes in two arguments:
* **initial_instance_count**: The number of deployed instances (1).
* **instance_type**: The type of SageMaker instance for deployment.

Note: If you run into an instance error, it may be because you chose the wrong training or deployment instance_type. It may help to refer to your previous exercise code to see which types of instances we used.

In [9]:
%%time

# deploy your model to create a predictor

from sagemaker.pytorch import PyTorchModel
model = PyTorchModel(model_data = estimator.model_data, role = role, framework_version = '0.4.0', entry_point = 'predict.py', source_dir = 'source_pytorch')
predictor = model.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

-------------------!CPU times: user 513 ms, sys: 19.1 ms, total: 532 ms
Wall time: 9min 32s


---
# Evaluating Your Model

Once your model is deployed, you can see how it performs when applied to our test data.

The provided cell below, reads in the test data, assuming it is stored locally in `data_dir` and named `test.csv`. The labels and features are extracted from the `.csv` file.

In [10]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import os

# read in test data, assuming it is stored locally
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]

In [11]:
print(f'test_x.shape={test_x.shape}')
print(f'test_y.size={test_y.size}')

test_x.shape=(25, 5)
test_y.size=25


In [12]:
test_x.values

array([[1.        , 0.96410256, 0.92227979, 0.87958115, 0.82075472],
       [0.76530612, 0.66438356, 0.58965517, 0.52083333, 0.62171053],
       [0.88444444, 0.34080717, 0.18099548, 0.11872146, 0.59745763],
       [0.61904762, 0.11229947, 0.04324324, 0.01092896, 0.42783505],
       [0.92      , 0.56164384, 0.3943662 , 0.27536232, 0.775     ],
       [0.99267399, 0.98154982, 0.9739777 , 0.96629213, 0.99305556],
       [0.41269841, 0.03278689, 0.        , 0.        , 0.34666667],
       [0.46268657, 0.01005025, 0.        , 0.        , 0.18932039],
       [0.58115183, 0.03703704, 0.        , 0.        , 0.24742268],
       [0.58421053, 0.0106383 , 0.        , 0.        , 0.29441624],
       [0.56637168, 0.03603604, 0.        , 0.        , 0.25833333],
       [0.48148148, 0.06766917, 0.02290076, 0.        , 0.27891156],
       [0.61979167, 0.1       , 0.02659574, 0.01612903, 0.34158416],
       [0.92173913, 0.77631579, 0.65486726, 0.54910714, 0.92941176],
       [1.        , 0.96153846, 0.

In [13]:
test_y.values

array([1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.,
       1., 0., 1., 0., 1., 1., 0., 0.])

## EXERCISE: Determine the accuracy of your model

Use your deployed `predictor` to generate predicted, class labels for the test data. Compare those to the *true* labels, `test_y`, and calculate the accuracy as a value between 0 and 1.0 that indicates the fraction of test data that your model classified correctly. You may use [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) for this calculation.

**To pass this project, your model should get at least 90% test accuracy.**

In [14]:
preds = predictor.predict(test_x.values)
preds

array([1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 0, 0])

In [15]:
# First: generate predicted, class labels

import numpy as np

test_y_preds = predictor.predict(test_x.values)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that your model generates the correct number of labels
assert len(test_y_preds)==len(test_y), 'Unexpected number of predictions.'
print('Test passed!')

Test passed!


In [16]:
# Second: calculate the test accuracy

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_y, test_y_preds)
print(f'accuracy={accuracy}')


## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_y_preds)
print('\nTrue class labels: ')
print(test_y.values)

accuracy=1.0

Predicted class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]

True class labels: 
[1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0.
 0.]


### Question 1: How many false positives and false negatives did your model produce, if any? And why do you think this is?

**Answer**: 

**How many false positives and false negatives did your model produce, if any?**

My model is `100%` accurate. Perfect score.
Hence, my model has `0` false positives and `0` false negatives.

**And why do you think this is?**

My model is highly successful because...

---

Initially, I computed the Longest Common Subsequence (LCS) and the containments of n-grams for `n = [1, 2, 3, 4, 5, 6, 7]`, a total of 8 features: `c_1`, `c_2`, `c_3`, `c_4`, `c_5`, `c_6`, `c_7`, and `lcs_word`.

Then I observed that:
- `c_2` was too correlated to `c_1` and `c_3`;
- `c_4` was too correlated to `c_3` and `c_5`;
- `c_6` was too correlated to `c_5` and `c_7`.

So, I decided to eliminate `c_2`, `c_4`, and `c_6`. Because they were too redundant, making machine learning models unnecessarily complicated.

I also noticed that:
- `c_1` and `c_3` were not so correlated;
- `c_3` and `c_5` were not so correlated;
- `c_5` and `c_7` were not so correlated;
- `lcs_word` was not so correlated to `c_1`, `c_3`, `c_5`, and `c_7`.

So, I decided to keep `c_1`, `c_3`, `c_5`, `c_7`, and `lcs_word`.
Because they represent a diversity of opinions that create a very accurate feature space.
In fact, the results were amazing: `100%` accuracy in test dataset. Perfect score.

Why are these features so successful?
- `c_1` is the vocabulary feature that represent single words.
- `c_7` represents the common long phrases.
- `c_3` and `c_5` cover the middle range of combinations of words.
- And `lcs_word` is a very accurate metric to predict copy/paste plagiarism and paraphrasing plagiarism.


### Question 2: How did you decide on the type of model to use? 

**Answer**:

I already explained above why I decided to keep the features `c_1`, `c_3`, `c_5`, `c_7`, and `lcs_word`. And why I decided to eliminate the features `c_2`, `c_4`, and `c_6`.

And I decided to use neural networks to classify these patterns because they are highly successful. In fact, Cybenko's universal approximation theorem <https://en.wikipedia.org/wiki/Universal_approximation_theorem> states that a neural network with an input layer, a hidden layer with sufficient hidden neurons, and an output layer can approximate any mapping. And indeed, my feedforward neural network with 5 input neurons, 200 hidden neurons, and 1 output neuron (plagiarized or not) successfully approximated this mapping, in just 300 epochs of training. Training was super fast (few minutes) due to the low complexity of this neural network and the small amount of data.

I used PyTorch because this deep learning framework is very easy to use and very Pythonic, in other words, very compatible with the philosophy of Python and numpy.

----
## EXERCISE: Clean up Resources

After you're done evaluating your model, **delete your model endpoint**. You can do this with a call to `.delete_endpoint()`. You need to show, in this notebook, that the endpoint was deleted. Any other resources, you may delete from the AWS console, and you will find more instructions on cleaning up all your resources, below.

In [17]:
# uncomment and fill in the line below!
predictor.delete_endpoint()


### Deleting S3 bucket

When you are *completely* done with training and testing models, you can also delete your entire S3 bucket. If you do this before you are done training your model, you'll have to recreate your S3 bucket and upload your training data again.

In [18]:
# deleting bucket, uncomment lines below

bucket_to_delete = boto3.resource('s3').Bucket(bucket)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': 'A67568BA5D6EC453',
   'HostId': 'B/mNZtaHBhJZb8A9s7uHyKA7L6St44EI7B41UiqxGYukw8FLhFOjfPXvYmuMHfUehJVH95HjIXQ=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'B/mNZtaHBhJZb8A9s7uHyKA7L6St44EI7B41UiqxGYukw8FLhFOjfPXvYmuMHfUehJVH95HjIXQ=',
    'x-amz-request-id': 'A67568BA5D6EC453',
    'date': 'Mon, 10 Feb 2020 08:59:46 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'plagiarism/train/.ipynb_checkpoints/test-checkpoint.csv'},
   {'Key': 'plagiarism/train/train.csv'},
   {'Key': 'plagiarism/train/.ipynb_checkpoints/train-checkpoint.csv'},
   {'Key': 'sagemaker-pytorch-2020-02-10-08-45-28-945/output/model.tar.gz'},
   {'Key': 'plagiarism/train/test.csv'},
   {'Key': 'sagemaker-pytorch-2020-02-10-08-45-28-945/debug-output/training_job_end.ts'},
   {'Key': 'sagemaker-pytorch-2020-02-10-08-50-12-066/sourcedir.ta

### Deleting all your models and instances

When you are _completely_ done with this project and do **not** ever want to revisit this notebook, you can choose to delete all of your SageMaker notebook instances and models by following [these instructions](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html). Before you delete this notebook instance, I recommend at least downloading a copy and saving it, locally.

---
## Further Directions

There are many ways to improve or add on to this project to expand your learning or make this more of a unique project for you. A few ideas are listed below:
* Train a classifier to predict the *category* (1-3) of plagiarism and not just plagiarized (1) or not (0).
* Utilize a different and larger dataset to see if this model can be extended to other types of plagiarism.
* Use language or character-level analysis to find different (and more) similarity features.
* Write a complete pipeline function that accepts a source text and submitted text file, and classifies the submitted text as plagiarized or not.
* Use API Gateway and a lambda function to deploy your model to a web application.

These are all just options for extending your work. If you've completed all the exercises in this notebook, you've completed a real-world application, and can proceed to submit your project. Great job!